In [ ]:
# This implementation of LDA leads to better results.

In [ ]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline

# Data Import and Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
df = pd.read_csv('./final_tweets_set.csv')
df

,Unnamed: 0,edit_history_tweet_ids,author_id,created_at,conversation_id,id,text,geo.place_id,referenced_tweets,city,inside,withheld.copyright,withheld.country_codes,withheld.scope,hashtag,text_hash_split,text_clean
0,1268,['1391443346443411457'],1267945445084598274,2021-05-09T17:22:00.000Z,1391443346443411457,1.391443e+18,Does #FortWorth ever cross your mind? \nhttps:...,NaN,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'Stockyards', 'SUV', 'Limo']",Does fort worth ever cross your mind? \nhttps:...,does fort worth ever cross your mind and 469 2...
1,4672,['1311653244112957440'],5445832,2020-10-01T13:04:37.000Z,1311653244112957440,1.311653e+18,P.F.L. Always! 2004-06\n.\n.\nFollow @playarab...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['playarabbit', 'theparksmall', 'fortworth', '...",P.F.L. Always! 2004-06\n.\n.\nFollow @playarab...,p f l always 06 follow play a rabbit the park ...
2,2327,['1423822631577456648'],41679815,2021-08-07T01:45:43.000Z,1423822631577456648,1.423823e+18,"Disabled vehicle, right lane blocked in #FortW...",42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'DFWTraffic']","Disabled vehicle, right lane blocked in fort w...",disabled vehicle right lane blocked in fort wo...
3,66,['1287040376738635776'],347029713,2020-07-25T15:01:52.000Z,1287040376738635776,1.287040e+18,Pickup Man the Cowboy’s Cowboy🤠 #stockyardsch...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['stockyardschampionshiprodeo', 'stockyardspro...",Pickup Man the Cowboy’s Cowboy🤠 stockyardscha...,pickup man the cowboys cowboy stockyardschampi...
4,2525,['1376576317903241217'],41679815,2021-03-29T16:45:44.000Z,1376576317903241217,1.376576e+18,Accident. two left lanes blocked. in #FortWort...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'DFWTraffic']",Accident. two left lanes blocked. in fort wort...,accident two left lanes blocked in fort worth ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27195,107618,['1315221808325435393'],1099150592138596353,2020-10-11T09:24:49.000Z,1315221808325435393,1.315222e+18,Wearing the smile my momma gave me 💕💖👄⚡️🍓🧠💅🏼🍇 ...,1279db2db14240db,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['SelfAddiction', 'LosAngeles']",Wearing the smile my momma gave me 💕💖👄⚡️🍓🧠💅🏼🍇 ...,wearing the smile my momma gave me self addict...
27196,108522,['1440294771709284363'],3526993403,2021-09-21T12:40:07.000Z,1440294771709284363,1.440295e+18,L.A. is Calling Again! 🎶 🎤 #singer #losangeles...,0051d76b1627f404,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['singer', 'losangeles', 'newjersey', 'sony', ...",L.A. is Calling Again! 🎶 🎤 singer los angeles ...,l a is calling again singer los angeles newjer...
27197,107637,['1314770648904994820'],32148675,2020-10-10T03:32:04.000Z,1314770648904994820,1.314771e+18,So get ready #LosAngeles!\nUnless #Miami #Heat...,35f9d631a3650f85,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['LosAngeles', 'Miami', 'Heat', 'LeBronJames',...",So get ready los angeles!\nUnless miami heat a...,so get ready los angeles unless miami heat att...
27198,107125,['1217178572462399489'],1196663730144075777,2020-01-14T20:16:00.000Z,1217178572462399489,1.217179e+18,That's the cop who threatened me worldstar @vi...,2da132a7bfebfc0c,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['pig', 'montebello', 'montebellopd', 'LosAnge...",That's the cop who threatened me worldstar @vi...,that s the cop who threatened me worldstar vic...


In [ ]:
# Convert to list
data = df.text_clean.values.tolist()
# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['does fort worth ever cross your mind and 469 265 4527 ready to show you the '
 'gateway to the west stockyards suv limo']


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['does', 'fort', 'worth', 'ever', 'cross', 'your', 'mind', 'and', 'ready', 'to', 'show', 'you', 'the', 'gateway', 'to', 'the', 'west', 'stockyards', 'suv', 'limo']]


In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [ ]:
! python -m spacy download en_core_web_md

2023-05-21 22:21:28.229626: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nltk.download('stopwords')
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'VERB']) #select noun and verb
print(data_lemmatized[:2])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['fort cross mind show gateway limo', 'follow play rabbit park fort rap hip']


# Creating the Model

In [ ]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
# minimum reqd occurences of a word 
                             stop_words='english',             
# remove stop words
                             lowercase=True,                   
# convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  
# num chars > 3
                             # max_features=50000,             
# max number of uniq words    
    )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               
# Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [ ]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
    evaluate_every=-1, learning_decay=0.7,
    learning_method='online', learning_offset=10.0,
    max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
    n_components=20, n_jobs=-1, perp_tol=0.1,
    random_state=100, topic_word_prior=None,
    total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)

In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -1370942.8687479354
Perplexity:  1208.2996398912996
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [ ]:
### THE FOLLOWING GRID SEARCH TAKES A LONG TIME. ONLY RUN FOR PARAMETER TUNING ONCE.
# # Define Search Param
# search_params = {'n_components': [5, 10, 15], 'learning_decay': [.5, .7, .9]}
# # Init the Model
# lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# # Init Grid Search Class
# model = GridSearchCV(lda, param_grid=search_params)
# # Do the Grid Search
# model.fit(data_vectorized)
# GridSearchCV(cv=None, error_score='raise',
#        estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
#              evaluate_every=-1, learning_decay=0.7, learning_method=None,
#              learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
#              mean_change_tol=0.001, n_components=None, n_jobs=1, perp_tol=0.1, random_state=None,
#              topic_word_prior=None, total_samples=1000000.0, verbose=0), n_jobs=1,
#        param_grid={'n_components': [5, 10, 15], 'learning_decay': [0.5, 0.7, 0.9]},
#        pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
#        scoring=None, verbose=0)

In [ ]:
# # Best Model
# #best_lda_model = model.best_estimator_
# best_lda_model = lda_model
# # Model Parameters
# print("Best Model's Params: ", model.best_params_)
# # Log Likelihood Score
# print("Best Log Likelihood Score: ", model.best_score_)
# # Perplexity
# print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

In [ ]:
# Create Document — Topic Matrix
best_lda_model = lda_model
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,6
Doc1,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.260000,0.010000,0.010000,0.130000,7
Doc2,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.670000,19
Doc3,0.860000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0
Doc4,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.880000,19
Doc5,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.150000,0.010000,0.010000,0.150000,0.580000,19
Doc6,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.890000,19
Doc7,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.810000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,8
Doc8,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.180000,0.010000,0.010000,0.010000,0.010000,0.170000,9
Doc9,0.010000,0.010000,0.010000,0.400000,0.010000,0.010000,0.010000,0.010000,0.500000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,8


In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names_out()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,abandon,accept,access,accessory,accident,accord,account,accounting,achieve,acre,...,year,yesterday,yoga,yorkcity,youth,youtube,youtuber,zone,zoo,zoom
Topic0,0.05,31.913131,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,...,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.05,0.05,0.05
Topic1,0.05,0.050000,0.05,0.05,0.05,0.05,0.05,0.05,12.245078,0.05,...,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.05,0.05,0.05
Topic2,0.05,0.050000,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,...,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.05,0.05,0.05
Topic3,0.05,0.050000,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,...,0.05,0.05,0.05,14.295375,0.05,0.05,0.05,0.05,0.05,0.05
Topic4,0.05,0.050000,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,...,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.05,0.05,0.05


In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=20)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,music,time,album,phoenix,producer,world,vibe,beat,songwriter,record,shoot,feel,singer,design,jam,bar,mixtape,piano,set,hype
Topic 1,day,today,team,photo,make,time,walk,hour,dream,gold,use,cut,hope,talk,color,include,fuck,spring,delivery,rise
Topic 2,art,night,game,artist,start,end,enjoy,view,model,eat,football,season,ticket,fan,sport,sit,drop,tattoo,project,face
Topic 3,come,work,miss,bring,police,report,shot,office,lose,cat,philadelphia,minute,client,contact,mile,com,bobtail,hate,steal,fort
Topic 4,make,stay,light,wait,tomorrow,care,kid,save,fitness,health,lead,update,pass,pen,heat,treat,gym,workout,arrive,finish
Topic 5,know,friend,video,trip,head,sign,rock,break,radio,like,star,podcast,drink,listen,goal,remember,tune,sky,return,nature
Topic 6,estate,way,watch,tour,sell,realtor,tell,listing,summit,mind,crew,buyer,property,agent,canon,ring,expect,root,seafood,cross
Topic 7,follow,check,food,share,place,play,park,music,beer,cost,summer,spot,turn,market,split,film,fly,hit,actor,coffee
Topic 8,city,tampa,year,tonight,support,thing,birthday,history,boy,mask,vote,quarter,wear,king,month,wish,eve,school,ice,kill
Topic 9,life,week,travel,home,visit,weekend,morning,family,join,house,raleigh,downtown,mattress,insta,chief,weather,sleep,club,catch,live


In [ ]:
# PICKING TOPICS
topics = ["music",                  #0
          "social",                 #1
          "social",                 #2
          "?",                      #3
          "social",                 #4
          "exploring",              #5
          "?",                      #6
          "social",                 #7
          "?",                      #8
          "everyday life",          #9
          "businesses/services",    #10
          "fun",                    #11
          "?",                      #12
          "explore",                #13
          "businesses/services",    #14
          "social",                 #15
          "music",                  #16
          "?",                      #17
          "sports",                 #18
          "transportation"]         #19
df_topic_keywords["Topics"]=topics
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,...,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19,Topics
Topic 0,music,time,album,phoenix,producer,world,vibe,beat,songwriter,record,...,feel,singer,design,jam,bar,mixtape,piano,set,hype,music
Topic 1,day,today,team,photo,make,time,walk,hour,dream,gold,...,cut,hope,talk,color,include,fuck,spring,delivery,rise,social
Topic 2,art,night,game,artist,start,end,enjoy,view,model,eat,...,season,ticket,fan,sport,sit,drop,tattoo,project,face,social
Topic 3,come,work,miss,bring,police,report,shot,office,lose,cat,...,minute,client,contact,mile,com,bobtail,hate,steal,fort,?
Topic 4,make,stay,light,wait,tomorrow,care,kid,save,fitness,health,...,update,pass,pen,heat,treat,gym,workout,arrive,finish,social
Topic 5,know,friend,video,trip,head,sign,rock,break,radio,like,...,podcast,drink,listen,goal,remember,tune,sky,return,nature,exploring
Topic 6,estate,way,watch,tour,sell,realtor,tell,listing,summit,mind,...,buyer,property,agent,canon,ring,expect,root,seafood,cross,?
Topic 7,follow,check,food,share,place,play,park,music,beer,cost,...,spot,turn,market,split,film,fly,hit,actor,coffee,social
Topic 8,city,tampa,year,tonight,support,thing,birthday,history,boy,mask,...,quarter,wear,king,month,wish,eve,school,ice,kill,?
Topic 9,life,week,travel,home,visit,weekend,morning,family,join,house,...,downtown,mattress,insta,chief,weather,sleep,club,catch,live,everyday life


In [ ]:
# Create Document — Topic Matrix
best_lda_model = lda_model
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic_num = np.argmax(df_document_topic.values, axis=1)
selected_topics = [topics[i] for i in dominant_topic_num]
df_document_topic['dominant_topic'] = selected_topics

df_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.01,0.01,0.01,0.01,0.01,0.01,0.51,0.01,0.01,0.01,...,0.17,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.17,?
Doc1,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.51,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.26,0.01,0.01,0.13,social
Doc2,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.17,0.67,transportation
Doc3,0.86,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,music
Doc4,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.88,transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc27195,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.26,0.26,...,0.01,0.01,0.01,0.01,0.01,0.01,0.26,0.01,0.01,?
Doc27196,0.35,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.35,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,music
Doc27197,0.01,0.01,0.01,0.12,0.12,0.01,0.01,0.01,0.12,0.01,...,0.01,0.12,0.01,0.01,0.01,0.01,0.01,0.34,0.12,sports
Doc27198,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.52,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,fun


In [ ]:
df['dominant_topic'] = selected_topics
df

,Unnamed: 0,edit_history_tweet_ids,author_id,created_at,conversation_id,id,text,geo.place_id,referenced_tweets,city,inside,withheld.copyright,withheld.country_codes,withheld.scope,hashtag,text_hash_split,text_clean,dominant_topic
0,1268,['1391443346443411457'],1267945445084598274,2021-05-09T17:22:00.000Z,1391443346443411457,1.391443e+18,Does #FortWorth ever cross your mind? \nhttps:...,NaN,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'Stockyards', 'SUV', 'Limo']",Does fort worth ever cross your mind? \nhttps:...,does fort worth ever cross your mind and 469 2...,?
1,4672,['1311653244112957440'],5445832,2020-10-01T13:04:37.000Z,1311653244112957440,1.311653e+18,P.F.L. Always! 2004-06\n.\n.\nFollow @playarab...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['playarabbit', 'theparksmall', 'fortworth', '...",P.F.L. Always! 2004-06\n.\n.\nFollow @playarab...,p f l always 06 follow play a rabbit the park ...,social
2,2327,['1423822631577456648'],41679815,2021-08-07T01:45:43.000Z,1423822631577456648,1.423823e+18,"Disabled vehicle, right lane blocked in #FortW...",42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'DFWTraffic']","Disabled vehicle, right lane blocked in fort w...",disabled vehicle right lane blocked in fort wo...,transportation
3,66,['1287040376738635776'],347029713,2020-07-25T15:01:52.000Z,1287040376738635776,1.287040e+18,Pickup Man the Cowboy’s Cowboy🤠 #stockyardsch...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['stockyardschampionshiprodeo', 'stockyardspro...",Pickup Man the Cowboy’s Cowboy🤠 stockyardscha...,pickup man the cowboys cowboy stockyardschampi...,music
4,2525,['1376576317903241217'],41679815,2021-03-29T16:45:44.000Z,1376576317903241217,1.376576e+18,Accident. two left lanes blocked. in #FortWort...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'DFWTraffic']",Accident. two left lanes blocked. in fort wort...,accident two left lanes blocked in fort worth ...,transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27195,107618,['1315221808325435393'],1099150592138596353,2020-10-11T09:24:49.000Z,1315221808325435393,1.315222e+18,Wearing the smile my momma gave me 💕💖👄⚡️🍓🧠💅🏼🍇 ...,1279db2db14240db,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['SelfAddiction', 'LosAngeles']",Wearing the smile my momma gave me 💕💖👄⚡️🍓🧠💅🏼🍇 ...,wearing the smile my momma gave me self addict...,?
27196,108522,['1440294771709284363'],3526993403,2021-09-21T12:40:07.000Z,1440294771709284363,1.440295e+18,L.A. is Calling Again! 🎶 🎤 #singer #losangeles...,0051d76b1627f404,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['singer', 'losangeles', 'newjersey', 'sony', ...",L.A. is Calling Again! 🎶 🎤 singer los angeles ...,l a is calling again singer los angeles newjer...,music
27197,107637,['1314770648904994820'],32148675,2020-10-10T03:32:04.000Z,1314770648904994820,1.314771e+18,So get ready #LosAngeles!\nUnless #Miami #Heat...,35f9d631a3650f85,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['LosAngeles', 'Miami', 'Heat', 'LeBronJames',...",So get ready los angeles!\nUnless miami heat a...,so get ready los angeles unless miami heat att...,sports
27198,107125,['1217178572462399489'],1196663730144075777,2020-01-14T20:16:00.000Z,1217178572462399489,1.217179e+18,That's the cop who threatened me worldstar @vi...,2da132a7bfebfc0c,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['pig', 'montebello', 'montebellopd', 'LosAnge...",That's the cop who threatened me worldstar @vi...,that s the cop who threatened me worldstar vic...,fun


In [ ]:
df.to_csv('./tweets_with_topics.csv', sep='\t')